# Signal Processing Guideline


In [ ]:
import os
import numpy as np
import quantities as pq
import matplotlib.pyplot as plt



## 1. Data Load

In [ ]:
from miv.io import load_data

In [ ]:
# Load dataset from OpenEphys recording
folder_path: str = "~/Open Ephys/2022-03-10-16-19-09"  # Data Path
dataset = load_data(folder_path, device="OpenEphys")

### 1.1. Meta Data Structure

In [ ]:
# Get signal and rate(hz)
#   signal     : np.array, shape(N, N_channels)
#   rate       : float
record_node: int = dataset.get_nodes[0]
recording = dataset[record_node]["experiment1"]["recording1"]
signal, _, rate = recording.continuous["100"]
# time = recording.continuous["100"].timestamp / rate
num_channels = signal.shape[1]

### 1.2. Array Data Structure

### 1.3 Raw Data

## 2. Filtering Raw Signal

We provide a set of signal-filter tools that can be pre-applied to the overall signal before the spike-detection stage. Here, we provide examples of how to easily create filter, or set of filters, and apply them to the dataset.

If you have further suggestion on other filters to include, please leave an issue on our [GitHub issue page](https://github.com/GazzolaLab/MiV-OS/issues) with `enhancement` tag.

In [ ]:
from miv.signal.filter import FilterCollection, ButterBandpass

### 2.1 Filter Collection

In [ ]:
# Butter bandpass filter
pre_filter = ButterBandpass(lowcut=300, highcut=3000, order=5)

# How to construct sequence of filters
pre_filter = (
    FilterCollection(tag="Filter Example")
        .append(ButterBandpass(lowcut=300, highcut=3000, order=5))
        #.append(Limiter(400*pq.mV))
        #.append(Filter1(**filter1_kwargs))
        #.append(Filter2(**filter2_kwargs))
)

### 2.2 Apply Filter

In [ ]:
# Apply filter to entire dataset
dataset.apply_filter(pre_filter)
filtered_signal = dataset[record_node]['experiment1']['recording1'].filtered_signal

# Apply filter to particular data in the dataset
#filtered_signal = dataset[record_node]['experiment1']['recording1'].apply_filter(pre_filter)

# Apply filter to array
rate = 30_000
filtered_signal = pre_filter(data_array, sampling_rate=rate)

# Retrieve data from dataset and apply filter
data = dataset[record_node]['experiment1']['recording1']
filtered_signal = pre_filter(data)

## 3. Spike Detection

In [ ]:
from miv.signal.spike import (
    compute_spike_threshold,
    detect_threshold_crossings,
    align_to_minimum,
)

In [ ]:
# Spike detection for each channel
spiketrain_list = []
for channel in range(num_channels):
    # Spike Detection: get spikestamp
    spike_threshold = compute_spike_threshold(signal)
    crossings = detect_threshold_crossings(signal, rate, spike_threshold, 0.003)
    spikes = align_to_minimum(signal, rate, crossings, 0.002)
    spikestamp = spikes / rate
    # Convert spikestamp to neo.SpikeTrain (for plotting)
    spiketrain = neo.SpikeTrain(spikestamp, units="sec")
    spiketrain_list.append(spiketrain)


## 4. Spike Visualization

In [ ]:
import neo
from viziphant.rasterplot import rasterplot_rates

In [ ]:
# Plot
rasterplot_rates(spiketrain_list)